# 加載


In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic Python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn.functional as F
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn as nn
import ipywidgets as widgets
from IPython.display import display
import csv
import ipywidgets as widgets
from jetbot import Robot
import numpy as np
from datetime import datetime
from jetbot import Camera, bgr8_to_jpeg
import torch
import cv2
import traitlets
import numpy as np
from jetbot import Camera, bgr8_to_jpeg
from torchvision import models, transforms
from ipywidgets import widgets
from IPython.display import display
import time
import torch
import cv2
import traitlets
import numpy as np
from jetbot import Camera, bgr8_to_jpeg
from torchvision import models, transforms
from ipywidgets import widgets
from IPython.display import display
import time
import ipywidgets

In [7]:
import time
import torch
import torchvision.transforms as transforms
import cv2
import numpy as np
from jetbot import Camera
import ipywidgets as widgets
from IPython.display import display

# 設定相機和 widget
camera = Camera.instance(width=224, height=224, fps=5)
widget_width = 224
widget_height = 224
image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)

# 顯示 widget
display(image_widget)

# 設定裝置 (若有 GPU 可用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 加載模型
model = torchvision.models.resnet34()
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("resnet34_final.pth"))
model = model.to(device)
model.eval()  # 將模型設定為推論模式

def execute(change):
    image = change['new']
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4691, 0.4032, 0.4579], [0.1740, 0.1485, 0.1688])
    ])
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(input_tensor)
        x, y = output[0].cpu().numpy()

    # 映射預測的 (x, y) 到圖片的像素坐標，假設 x 和 y 是 [0, 1] 範圍內的預測
    x_pixel =int( x* widget_width / 2 + widget_width / 2)
    y_pixel =int(y * widget_height / 2 + widget_height / 2)
    
    # 在圖片上繪製預測結果
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # 確保格式正確
    cv2.circle(image, (x_pixel, y_pixel), 5, (0, 255, 0), -1)  # 綠色點表示預測位置
    
    # 更新 widget 顯示處理後的圖片
    image_widget.value = bgr8_to_jpeg(image)

# 測試執行一次
execute({'new': camera.value})


Image(value=b'', format='jpeg', height='224', width='224')

cuda


# 開始偵測

In [8]:
# 監聽相機數據變化並執行預測
camera.observe(execute, names='value')

# 關閉偵測


In [9]:
camera.unobserve(execute, names='value')

In [6]:
camera.stop()